In [1]:
import pymc as pm
import numpy as np
import pytensor.tensor as pt
import pytensor
from pymc.logprob.basic import logp

In [2]:
x_rv = pt.expm1(pt.random.normal())

x_vv = x_rv.clone()

x_logp_fn = pytensor.function([x_vv], pt.sum(logp(x_rv, x_vv)))

In [2]:
import pytensor.tensor as pt
import pytensor

In [3]:
pt.log1pexp

Elemwise(scalar_op=scalar_softplus,inplace_pattern=<frozendict {}>)

In [11]:
def log(p = None):
    return np.log(1+p)
log(True)

0.6931471805599453

In [10]:
np.log(2)

0.6931471805599453

In [9]:
np.log(np.cosh(0.25))

0.030929803620161386

In [ ]:
y = np.exp(x)-1
y + 1 = np.exp(x)
np.log()

In [16]:
np.arccosh(10)

2.993222846126381

In [7]:

# get assertion error because we are caulting jac det directly therefore skipping this line
# return pt.switch(pt.isnan(jacobian), -np.inf, input_logprob + jacobian)
# so is returned as nan rather than -inf as is expected
# hence switch statement in the test

# also in have changed line 416 to 
# return pt.switch(pt.isnan(input_logprob + jacobian), -np.inf, input_logprob + jacobian)
# as cosh transform was returtuning nan due to input_logprob being nan


/var/folders/yw/hvmlf6sd7yv3040wn8str6540000gn/T/ipykernel_5322/294505405.py:1: RuntimeWarning: invalid value encountered in log
  np.log(-2)


nan

In [35]:
import scipy.stats as stats

def test_cosh_transform(test_val):
    x_rv = pt.abs(pt.random.normal())
    y_rv = pt.random.lognormal(sigma=1)

    x_vv = x_rv.clone()
    y_vv = y_rv.clone()
    x_transformed = pm.math.cosh(x_vv)
    y_transformed = pm.math.cosh(y_vv)

    x_logp_fn = pytensor.function([x_vv], joint_logprob({x_rv: x_transformed}, sum=False))
    y_logp_fn = pytensor.function([y_vv], joint_logprob({y_rv: y_transformed}, sum=False))

    # Calculate the log probability using SciPy's lognorm distribution
    scipy_logp = stats.lognorm.logpdf(test_val, 1)

    assert np.allclose(x_logp_fn(test_val), y_logp_fn(test_val))
    assert np.allclose(y_logp_fn(test_val), scipy_logp)

In [ ]:
import pytensor
import pytensor.tensor as at

k = at.iscalar("k")
A = at.vector("A")

# Symbolic description of the result
result, updates = pytensor.scan(fn=lambda prior_result, A: prior_result * A,
                              outputs_info=at.ones_like(A),
                              non_sequences=A,
                              n_steps=k)

# We only care about A**k, but scan has provided us with A**1 through A**k.
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.
final_result = result[-1]

# compiled function that returns A**k
power = pytensor.function(inputs=[A,k], outputs=final_result, updates=updates)

print(power(range(10),2))
print(power(range(10),4))

In [5]:
def erfcxinv(y, tol = 1e-10, max_iter=100):
    # Compute x using the appropriate formula for each value of y
    x = at.switch(y <= 1, 1. / (y * at.sqrt(np.pi)), -at.sqrt(at.log(y)))
    # for n in range(1, 8):
    #     x = x - (at.erfcx(x) - y) / (2 * x * at.erfcx(x) - 2 / at.sqrt(np.pi))
    iter_count = 0
    while iter_count < max_iter:
        iter_count += 1
        fx = at.erfcx(x) - y
        fpx = 2 * x * at.erfcx(x) - 2 / at.sqrt(np.pi)
        delta_x = fx / fpx
        x = x - delta_x
        if (at.abs(delta_x) < tol).all():
            break
    return x  


In [6]:
erfcxinv(np.array([0.5, 0.5]))

Elemwise{sub,no_inplace}.0